In [2]:
import polars as pl
import pathlib

In [3]:
csvFile = "./data/input/traits_no_measurement.tsv"

df = pl.read_csv(csvFile, sep="\t")
df

trait,trait_uri,studies,association_counts,papers,Olga's comments
str,str,i64,i64,i64,str
"""clinical treat...","""http://www.ebi...",687,31,687,null
"""employment sta...","""http://www.ebi...",687,10,687,null
"""cortical thick...","""http://www.ebi...",325,1292,325,null
"""white matter i...","""http://www.ebi...",306,502,306,null
"""injury""","""http://www.ebi...",270,0,270,null
"""lymphocyte cou...","""http://www.ebi...",225,2729,225,null
"""type 2 diabete...","""http://purl.ob...",175,5003,175,"""Glucose homeos..."
"""body mass inde...","""http://www.ebi...",151,8912,151,"""Glucose homeos..."
"""encounter with...","""http://www.ebi...",148,0,148,null


rsID,Gene,Ref allele,Alt allele,Risk allele,Genotype,Conclusion,Weight,PMID,Population,GWAS study design,P-value
str,str,str,str,str,str,str,str,str,str,str,str
"""rs6025""","""F5""","""C""","""T""","""T""","""CC""","""The F5 gene en...","""0""","""[PMID 28373160...","""European; Afri...","""[PMID: 3167686...","""[PMID: 3167686..."
"""rs6025""","""F5""","""C""","""T""","""T""","""CT""","""The F5 gene en...","""-1,5""","""[PMID 28373160...","""European; Afri...","""[PMID: 3167686...","""[PMID: 3167686..."
"""rs6025""","""F5""","""C""","""T""","""T""","""TT""","""The F5 gene en...","""-2""","""[PMID 28373160...","""European; Afri...","""[PMID: 3167686...","""[PMID: 3167686..."
"""rs1799963""","""F2""","""G""","""A""","""A""","""GG""","""G20210A polymo...","""0""","""[PMID 23900608...","""European""","""6,135 European...","""[PMID: 2690860..."
"""rs1799963""","""F2""","""G""","""A""","""A""","""GA""","""G20210A polymo...","""-1,3""","""[PMID 23900608...","""European""","""6,135 European...","""[PMID: 2690860..."
"""rs1799963""","""F2""","""G""","""A""","""A""","""AA""","""G20210A polymo...","""-1,8""","""[PMID 23900608...","""European""","""6,135 European...","""[PMID: 2690860..."
"""rs1800790""","""FGB""","""G""","""A""","""A""","""GG""","""FGB gene encod...","""0""",""" [PMID 2923550...",null,null,null
"""rs1800790""","""FGB""","""G""","""A""","""A""","""GA""","""FGB gene encod...","""-0,3""",""" [PMID 2923550...",null,null,null
"""rs1800790""","""FGB""","""G""","""A""","""A""","""AA""","""FGB gene encod...","""-0,7""",""" [PMID 2923550...",null,null,null
